In [1]:
import sys
sys.path.append('..')

from src.tool.tool_registry import ToolRegistry
from langchain_core.messages import SystemMessage, AIMessage, HumanMessage
from langchain_openai import ChatOpenAI
from langchain.agents import create_agent

model = ChatOpenAI(
    model="qwen3-8B", 
    base_url="http://127.0.0.1:1234/v1", 
    api_key=""
)

tool_registry = ToolRegistry()
tools = tool_registry.get_tools()

agent_executor = create_agent(model, tools)

In [2]:
chat_history = [
    SystemMessage(
    """
        You are an expert data analyst with enhanced RAG capabilities. You will answer user questions by create and executing Cypher queries to Database.

        You have 5 tools to help you create a correct, efficient, and precise Cypher query based on user questions.
        1. entity_extraction: Extracts medical entities and relationships from user input
        2. rag_enhanced_search: Performs semantic search across medical entities
        3. context_builder: Combines query, entities, and RAG results into enriched context
        4. schema_linking: Provides graph database schema definition
        5. execute_cypher: Executes Cypher queries against the graph database

        Step-by-step
        1. Extract medical entities from user question using entity_extraction
        2. Use rag_enhanced_search with extracted entities to find similar items in the database
        3. Use schema_linking to understand the database structure
        4. Use context_builder to create enriched context for query generation
        5. Finally, use execute_cypher with the enriched context to get precise results

        You can skip any tool if not needed.
        Summary the final answer based on the query result and provide clear explanation.
    """,
    ),
]

In [3]:
from langchain_core.callbacks import BaseCallbackHandler

class ToolExecutionPrinter(BaseCallbackHandler):
    def on_tool_start(self, serialized, input_str, **kwargs):
        """Run when a tool starts running."""
        tool_name = serialized.get("name")
        print(f"\n[LOG] 🛠️  Agent is entering tool: {tool_name}")
        print(f"[LOG]    Input args: {input_str}\n")

    def on_tool_end(self, output, **kwargs):
        """Run when a tool ends running."""
        print(f"[LOG] ✅ Tool execution finished.\n")

In [4]:
import re

def clean_llm_response(content: str) -> str:
    """Clean LLM response by removing thinking tags and unwanted content."""
    
    # 1. Remove <think>...</think> tags and their content
    content = re.sub(r'<think>.*?</think>', '', content, flags=re.DOTALL)
    
    # 2. Remove markdown code blocks
    # Note: It's safer to use replace or regex than hard slicing indices
    if content.startswith("```json"):
        content = content.replace("```json", "", 1)
    if content.startswith("```"):
        content = content.replace("```", "", 1)
    if content.endswith("```"):
        content = content[:-3]
        
    # 3. Remove any other XML-like tags
    content = re.sub(r'<[^>]+>', '', content)
    
    return content.strip()

In [5]:
user_input = """
berapa banyak claim untuk heart attack
"""

message = [
    HumanMessage(f"This is the question: {user_input}"),
    AIMessage("Understood. I will follow the workflow to answer the question."),
    HumanMessage("Detect what language is the question input and respond in the same language."),
    AIMessage("Understood. I will detect the input language and respond accordingly.")
]

final_messages = chat_history + message

response = agent_executor.invoke({
    "messages": final_messages,
})

raw_content = response['messages'][-1].content

# 3. Apply the Cleaning Function
final_output = clean_llm_response(raw_content)

# 4. Print the Cleaned Response
print(f"Response: {final_output}")

Received notification from DBMS server: <GqlStatusObject gql_status='01N51', status_description='warn: relationship type does not exist. The relationship type `TREATED_WITH` does not exist in database `neo4j`. Verify that the spelling is correct.', position=<SummaryInputPosition line=1, column=39, offset=38>, raw_classification='UNRECOGNIZED', classification=<NotificationClassification.UNRECOGNIZED: 'UNRECOGNIZED'>, raw_severity='WARNING', severity=<NotificationSeverity.WARNING: 'WARNING'>, diagnostic_record={'_classification': 'UNRECOGNIZED', '_severity': 'WARNING', '_position': {'offset': 38, 'line': 1, 'column': 39}, 'OPERATION': '', 'OPERATION_CODE': '0', 'CURRENT_SCHEMA': '/'}> for query: "MATCH (d:Diagnosis {code: 'I21.9'})-[:TREATED_WITH]->(c:Claim) RETURN COUNT(c) AS total_claims"


Response: There are **4 claims** related to heart attacks (ICD-10 code I21.9: "Acute Myocardial Infarction") in the dataset. 

This count reflects all medical claims coded with this specific diagnosis. Let me know if you'd like to explore claims for other conditions or analyze additional details!


In [6]:
user_input = """
berapa banyak claim untuk serangan jantung
"""

message = [
    HumanMessage(f"This is the question: {user_input}"),
    AIMessage("Understood. I will follow the workflow to answer the question."),
    HumanMessage("Detect what language is the question input and respond in the same language."),
    AIMessage("Understood. I will detect the input language and respond accordingly.")
]

final_messages = chat_history + message

response = agent_executor.invoke({
    "messages": final_messages,
})

raw_content = response['messages'][-1].content

# 3. Apply the Cleaning Function
final_output = clean_llm_response(raw_content)

# 4. Print the Cleaned Response
print(f"Response: {final_output}")

Response: There are **4 claims** associated with the diagnosis **I21.9 (Acute Myocardial Infarction)** in the database.

This count specifically reflects claims coded with the exact ICD-10 code for heart attacks. If you'd like to include related diagnoses (e.g., other heart attack subtypes), let me know and I can expand the search!
